In [4]:
!git add .

^C


# SparkSQL and DataFrames 

<a href = "http://yogen.io"><img src="http://yogen.io/assets/logo.svg" alt="yogen" style="width: 200px; float: right;"/></a>

## RDDs, DataSets, and DataFrames

RDDs are the original interface for Spark programming.

DataFrames were introduced in 1.3

Datasets were introduced in 1.6, and unified with DataFrames in 2.0

### Advantages of DataFrames:

from https://www.datacamp.com/community/tutorials/apache-spark-python:

> More specifically, the performance improvements are due to two things, which you’ll often come across when you’re reading up DataFrames: custom memory management (project Tungsten), which will make sure that your Spark jobs much faster given CPU constraints, and optimized execution plans (Catalyst optimizer), of which the logical plan of the DataFrame is a part.

## SparkSQL and DataFrames 


pyspark does not have the Dataset API, which is available only if you use Spark from a statically typed language: Scala or Java.

From https://spark.apache.org/docs/2.2.0/sql-programming-guide.html:

> A DataFrame is a Dataset organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in R/Python, but with richer optimizations under the hood. DataFrames can be constructed from a wide array of sources such as: structured data files, tables in Hive, external databases, or existing RDDs. The DataFrame API is available in Scala, Java, Python, and R. In Scala and Java, a DataFrame is represented by a Dataset of Rows. In the Scala API, DataFrame is simply a type alias of Dataset[Row]. While, in Java API, users need to use Dataset&lt;Row> to represent a DataFrame.


### The pyspark.sql module

Important classes of Spark SQL and DataFrames:

* `pyspark.sql.SparkSession` Main entry point for DataFrame and SQL functionality.

* `pyspark.sql.DataFrame` A distributed collection of data grouped into named columns.

* `pyspark.sql.Column` A column expression in a DataFrame.

* `pyspark.sql.Row` A row of data in a DataFrame.

* `pyspark.sql.GroupedData` Aggregation methods, returned by DataFrame.groupBy().

* `pyspark.sql.DataFrameNaFunctions` Methods for handling missing data (null values).

* `pyspark.sql.DataFrameStatFunctions` Methods for statistics functionality.

* `pyspark.sql.functions` List of built-in functions available for DataFrame.

* `pyspark.sql.types` List of data types available.

* `pyspark.sql.Window` For working with window functions.

http://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

## SparkSession

The traditional way to interact with Spark is the SparkContext. In the notebooks we get that from the pyspark driver.

From 2.0 we can use SparkSession to replace SparkConf, SparkContext and SQLContext

In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [5]:
from pyspark.sql import SparkSession
session = SparkSession.builder.getOrCreate()
session.sparkContext

<SparkContext master=local[*] appName=PySparkShell>

#### Passing other options to spark session:
    
    

In [6]:
session = SparkSession.builder.config('comeoption.key','somevalue').getOrCreate()

### Creating DataFrames

SparkSession.createDataFrame: from an RDD, a list or a pandas.DataFrame.

In [7]:
import random

In [8]:
random.seed(42)
ids=range(12)
positions = [random.choice(['mechanic','sales','manager']) for id_ in ids]
positions

rows = zip(ids,positions)

df = session.createDataFrame(rows)
df.show(5)

+---+--------+
| _1|      _2|
+---+--------+
|  0| manager|
|  1|mechanic|
|  2|mechanic|
|  3| manager|
|  4|   sales|
+---+--------+
only showing top 5 rows



### Creating DataFrames

* From RDDs
* from Hive tables
* From Spark sources: parquet (default), json, jdbc, orc, libsvm, csv, text


#### From RDDs

In [27]:
rdd = session.sparkContext.textFile('../../../../../Data/coupon150720.csv')
rdd.take(5)
session.createDataFrame(zip(ids,positions),schema=['ids','positions'])
split_rdd = rdd.map(lambda line:line.split(','))
new_df = session.createDataFrame(split_rdd)
new_df

DataFrame[_1: string, _2: string, _3: string, _4: string, _5: string, _6: string, _7: string, _8: string, _9: string, _10: string, _11: string, _12: string, _13: string, _14: string, _15: string]

In [30]:
from pyspark.sql import types


In [31]:
fields = [types.StructField('id',types.IntegerType()),
         types.StructField('position',types.StringType())]

my_schema = types.StructType(fields)
my_schema
df1 = session.createDataFrame(zip(ids,positions),schema=my_schema)

In [36]:
df_from_csv = session.read.csv('../../../../../Data/coupon150720.csv')
df_from_csv.show(5)

+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|           _c0|_c1|_c2|_c3|_c4|_c5|   _c6|_c7|_c8|_c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
|79062005698500|  1|MAA|AUH| 9W| 9W| 56.79|USD|  1|  H|   H|0526|150904|  OK|IAF0|
|79062005698500|  2|AUH|CDG| 9W| 9W| 84.34|USD|  1|  H|   H|6120|150905|  OK|IAF0|
|79062005924069|  1|CJB|MAA| 9W| 9W|  60.0|USD|  1|  H|   H|2768|150721|  OK|IAA0|
|79065668570385|  1|DEL|DXB| 9W| 9W|160.63|USD|  2|  S|   S|0546|150804|  OK|INA0|
|79065668737021|  1|AUH|IXE| 9W| 9W|152.46|USD|  1|  V|   V|0501|150803|  OK|INA0|
+--------------+---+---+---+---+---+------+---+---+---+----+----+------+----+----+
only showing top 5 rows



In [37]:
df_with_sql = session.sql('SELECT * FROM csv.`../../../../../Data/coupon150720.csv`')
df_with_sql.show()

+--------------+---+---+---+----+----+------+---+---+----+----+----+------+----+----+
|           _c0|_c1|_c2|_c3| _c4| _c5|   _c6|_c7|_c8| _c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+----+----+------+---+---+----+----+----+------+----+----+
|79062005698500|  1|MAA|AUH|  9W|  9W| 56.79|USD|  1|   H|   H|0526|150904|  OK|IAF0|
|79062005698500|  2|AUH|CDG|  9W|  9W| 84.34|USD|  1|   H|   H|6120|150905|  OK|IAF0|
|79062005924069|  1|CJB|MAA|  9W|  9W|  60.0|USD|  1|   H|   H|2768|150721|  OK|IAA0|
|79065668570385|  1|DEL|DXB|  9W|  9W|160.63|USD|  2|   S|   S|0546|150804|  OK|INA0|
|79065668737021|  1|AUH|IXE|  9W|  9W|152.46|USD|  1|   V|   V|0501|150803|  OK|INA0|
|79062006192650|  1|RPR|BOM|  9W|  9W|  68.5|USD|  1|   K|   K|2202|150720|  OK|IAE0|
|79062006192650|  2|BOM|RPR|  9W|  9W|  68.5|USD|  1|   H|   H|0377|150721|  OK|IAE0|
|79062005733853|  1|DEL|DED|  9W|  9W| 56.16|USD|  1|   V|   V|2839|150801|  OK|INA0|
|79062005836987|  1|ATL|LGA|  AA|  AA|  28.3|USD|  1| 

### Inferring and specifying schemas

#### Fully specifying a schema

We need to create a `StructType` composed of `StructField`s. each of those specifies afiled with name, type and `nullable` properties. 

#### From csv files

We can either read them directly into dataframes or read them as RDDs and transform that into a DataFrame. This second way will be very useful if we have unstructured data like web server logs.

#### From other types of data

Apache Parquet is a free and open-source column-oriented data store of the Apache Hadoop ecosystem. It is similar to the other columnar storage file formats available in Hadoop namely RCFile and Optimized RCFile. It is compatible with most of the data processing frameworks in the Hadoop environment.

### Basic operations with DataFrames

### Filtering and selecting

Syntax inspired in SQL.

In [41]:
df1.select('id')


+---+--------+
| id|position|
+---+--------+
|  0| manager|
|  1|mechanic|
|  2|mechanic|
|  3| manager|
|  4|   sales|
|  5|mechanic|
|  6|mechanic|
|  7|mechanic|
|  8| manager|
|  9|mechanic|
| 10| manager|
| 11| manager|
+---+--------+



In [55]:
df1[df1['position'] == 'manager'].show()

+---+--------+
| id|position|
+---+--------+
|  0| manager|
|  3| manager|
|  8| manager|
| 10| manager|
| 11| manager|
+---+--------+



In [50]:
df1[df1['position'] == 'manager'].show()
df1.where(df1['position']=='manager').select('id').show()


+---+--------+
| id|position|
+---+--------+
|  0| manager|
|  3| manager|
|  8| manager|
| 10| manager|
| 11| manager|
+---+--------+

+---+
| id|
+---+
|  0|
|  3|
|  8|
| 10|
| 11|
+---+



`where` is exactly synonimous with `filter`

In [58]:
!git push

Counting objects: 13, done.
Delta compression using up to 4 threads.
Compressing objects: 100% (13/13), done.
Writing objects: 100% (13/13), 2.82 MiB | 1.64 MiB/s, done.
Total 13 (delta 7), reused 0 (delta 0)
remote: Resolving deltas: 100% (7/7), completed with 6 local objects.
To https://github.com/hguadalupe/master-data-science.git
   74c1024..7dbddbc  master -> master


#### Exercise

Extract all employee ids which correspond to managers

### Adding columns

Dataframes are immutable, since they are built on top of RDDs, so we not assing to them. We need to create new RDDS with the appropriate columns.

### User defined functions

Creating `udf`s allows us to "vectorize" operations: write a standard function to process single elements, then build a udf with that that works on columns in a DataFrame, like a SQL function.

In [2]:
from pyspark.sql import functions

If we want the resulting columns to be of a particular type, we need to specify the return type. This is because in Python return types can not be inferred.

In [61]:
df1.select(functions.sqrt(df1['id'])).show()

+------------------+
|          SQRT(id)|
+------------------+
|               0.0|
|               1.0|
|1.4142135623730951|
|1.7320508075688772|
|               2.0|
|  2.23606797749979|
| 2.449489742783178|
|2.6457513110645907|
|2.8284271247461903|
|               3.0|
|3.1622776601683795|
|   3.3166247903554|
+------------------+



In [ ]:
udf_log1p = functions.udf(math.log)

In [62]:
df1=df.select('id',
             'position',
             lambda col: col*2)

AttributeError: 'function' object has no attribute '_get_object_id'

Think about this function: what is its return type?

#### Exercise: 

Create a 'salary' field in our df. make it 30000 for mechanics, 40000 for salespeople and 70000 for managers.



In [28]:
def assign_salary(position):
    salaries={'manager':70000,'sales':40000,'mechanic':30000}
    return salaries.get(position,'cuc')

assign_salary('mana')

salary = functions.udf(assign_salary)
df1.select('id','position',assign_salary('position')).show()

NameError: name 'df1' is not defined

In [32]:
def assign_salary(position):
    if role == 'manager':
        return 70000
    elif role == 'mechanic':
        return 30000
    else:
        return 40000
    
salary = functions.udf(assign_salary)
salary

<function __main__.assign_salary>

### Summary statistics

https://databricks.com/blog/2015/06/02/statistical-and-mathematical-functions-with-dataframes-in-spark.html

### .crosstab()

### Grouping

### Intersections

Ver much like SQL joins.

In [78]:
data = list(zip([10,11,12,10,12,20,11,23,40],
           [5000,10000,2000,0,1000,500,3000,2000,2000],
           [random.choice(['madrid','barcelona','sevilla']) for _ in range(9)]))

data2 = list(zip([10,11,12,10,12],
           [5000,10000,2000,0,1000],
           [random.choice(['madrid','barcelona','sevilla']) for _ in range(5)]))



In [77]:
new_df = session.createDataFrame(data,schema=['id','salary','city'])
new_df2 = session.createDataFrame(data2,schema=['id','bonus','position']).cache()

In [74]:
new_df.show()

+---+-----+---------+
| id|bonus| position|
+---+-----+---------+
| 10| 5000|barcelona|
| 11|10000|   madrid|
| 12| 2000|  sevilla|
| 10|    0|barcelona|
| 12| 1000|  sevilla|
| 20|  500|  sevilla|
| 11| 3000|barcelona|
| 23| 2000|  sevilla|
| 40| 2000|   madrid|
+---+-----+---------+



In [66]:
new3 = new_df.join(new_df2)
new3.show()

AnalysisException: 'Detected cartesian product for INNER join between logical plans\nInMemoryRelation [id#277L, bonus#278L, position#279], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)\n   +- Scan ExistingRDD[id#277L,bonus#278L,position#279]\nand\nInMemoryRelation [id#299L, bonus#300L, position#301], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)\n   +- Scan ExistingRDD[id#299L,bonus#300L,position#301]\nJoin condition is missing or trivial.\nUse the CROSS JOIN syntax to allow cartesian products between these relations.;'

In [42]:
sc.uiWebUrl

'http://10.0.2.15:4040'

#### Exercise

Calculate the [z-score](http://www.statisticshowto.com/probability-and-statistics/z-score/) of each employee's salary for their location

In [48]:
new_df2.show()

AttributeError: 'function' object has no attribute 'show'

We can do each aggregation as in individual step, with a number of different syntaxes:

Or we can use the built-in functions defined in [the `pyspark.sql` module](http://spark.apache.org/docs/2.2.0/api/python/pyspark.sql.html#module-pyspark.sql.functions)

Once calculated, we annotate each employee with the values for their location and calculate their z-score

Note that we can build more complex boolean conditions for joining, as well as joining on columns that do not have the same name:

### Handling null values

## SQL querying

We need to register our DataFrame as a table in the SQL context in order to be able to query against it.

Once registered, we can perform queries as complex as we want.

#### Exercise:

replicate the previous exercise, but with SparkSQL instead of dataframe methods.

In [79]:
new_df.show()

+---+------+---------+
| id|salary|     city|
+---+------+---------+
| 10|  5000|barcelona|
| 11| 10000|   madrid|
| 12|  2000|  sevilla|
| 10|     0|barcelona|
| 12|  1000|  sevilla|
| 20|   500|  sevilla|
| 11|  3000|barcelona|
| 23|  2000|  sevilla|
| 40|  2000|   madrid|
+---+------+---------+



In [92]:
df1 = new_df.groupby('city').agg(functions.mean('salary'),functions.stddev_pop('salary'))
df1.show()

+---------+------------------+------------------+
|     city|       avg(salary)|stddev_pop(salary)|
+---------+------------------+------------------+
|  sevilla|            1375.0|  649.519052838329|
|   madrid|            6000.0|            4000.0|
|barcelona|2666.6666666666665|2054.8046676563254|
+---------+------------------+------------------+



In [99]:
joineddf = new_df.join(df1, 'city' )
joineddf.show()

+---------+---+------+------------------+------------------+
|     city| id|salary|       avg(salary)|stddev_pop(salary)|
+---------+---+------+------------------+------------------+
|  sevilla| 12|  2000|            1375.0|  649.519052838329|
|  sevilla| 12|  1000|            1375.0|  649.519052838329|
|  sevilla| 20|   500|            1375.0|  649.519052838329|
|  sevilla| 23|  2000|            1375.0|  649.519052838329|
|   madrid| 11| 10000|            6000.0|            4000.0|
|   madrid| 40|  2000|            6000.0|            4000.0|
|barcelona| 10|  5000|2666.6666666666665|2054.8046676563254|
|barcelona| 10|     0|2666.6666666666665|2054.8046676563254|
|barcelona| 11|  3000|2666.6666666666665|2054.8046676563254|
+---------+---+------+------------------+------------------+



In [105]:
result = joineddf.select('id','city','salary').((joineddf ['salary'] - joineddf ['salary']) / 
                                              joineddf ['stddev_pop(salary)']).alias('z-score')
result.show()

SyntaxError: invalid syntax (<ipython-input-105-67827e98e489>, line 1)

## Interoperation with Pandas

Easy peasy. We can convert a spark DataFrame into a Pandas one, which will `collect` it, and viceversa, which will distribute it.

## Writing out


In [12]:
raw_coupons = session.read.csv('../../../../../Data/coupon150720.csv')

In [35]:
raw_coupons.show(600)

+--------------+---+---+---+----+----+------+---+---+----+----+----+------+----+----+
|           _c0|_c1|_c2|_c3| _c4| _c5|   _c6|_c7|_c8| _c9|_c10|_c11|  _c12|_c13|_c14|
+--------------+---+---+---+----+----+------+---+---+----+----+----+------+----+----+
|79062005698500|  1|MAA|AUH|  9W|  9W| 56.79|USD|  1|   H|   H|0526|150904|  OK|IAF0|
|79062005698500|  2|AUH|CDG|  9W|  9W| 84.34|USD|  1|   H|   H|6120|150905|  OK|IAF0|
|79062005924069|  1|CJB|MAA|  9W|  9W|  60.0|USD|  1|   H|   H|2768|150721|  OK|IAA0|
|79065668570385|  1|DEL|DXB|  9W|  9W|160.63|USD|  2|   S|   S|0546|150804|  OK|INA0|
|79065668737021|  1|AUH|IXE|  9W|  9W|152.46|USD|  1|   V|   V|0501|150803|  OK|INA0|
|79062006192650|  1|RPR|BOM|  9W|  9W|  68.5|USD|  1|   K|   K|2202|150720|  OK|IAE0|
|79062006192650|  2|BOM|RPR|  9W|  9W|  68.5|USD|  1|   H|   H|0377|150721|  OK|IAE0|
|79062005733853|  1|DEL|DED|  9W|  9W| 56.16|USD|  1|   V|   V|2839|150801|  OK|INA0|
|79062005836987|  1|ATL|LGA|  AA|  AA|  28.3|USD|  1| 

In [46]:
raw_coupons.registerTempTable("records")
ndf = session.sql('select _c0, _c1, _c2, _c3, _c4, _c5, _c6 from records where _c3 = "MAD"').show()

+--------------+---+---+---+---+---+------+
|           _c0|_c1|_c2|_c3|_c4|_c5|   _c6|
+--------------+---+---+---+---+---+------+
|79062005639642|  4|BRU|MAD| UX| UX| 21.02|
|79065668754871|  6|BRU|MAD| SN| 9W| 27.66|
|79065668917696|  2|CDG|MAD| AF| AF| 46.97|
|79062006133090|  2|CDG|MAD| AF| AF|  3.38|
|79062006110497|  1|CDG|MAD| AF| AF| 26.02|
|79062006110505|  1|CDG|MAD| AF| AF| 26.02|
|79062006137072|  3|ORY|MAD| UX| AF|  87.2|
|79062006116586|  2|CDG|MAD| AF| AF| 31.51|
|79062006128336|  4|CDG|MAD| AF| AF| 26.63|
|79062006163025|  2|JFK|MAD| DL| AF|110.28|
|79062006146358|  2|PRG|MAD| IB| OK| 72.03|
|79062006146359|  2|PRG|MAD| IB| OK| 72.03|
|79062006025679|  1|PRG|MAD| OK| OK| 82.16|
|79062006015995|  4|CDG|MAD| AF| AF|158.96|
|79062006060426|  1|ORY|MAD| UX| AF|141.23|
|79062006070095|  1|ORY|MAD| UX| AF| 85.53|
|79062006074377|  1|CDG|MAD| AF| AF| 85.53|
|79062006078177|  2|CDG|MAD| AF| AF|  9.11|
|79062006080228|  2|CDG|MAD| AF| AF| 20.25|
|79062005980557|  1|CDG|MAD| AF|

#### Exercise

Repeat the exercise from the previous notebook, but this time with DataFrames.

Get stats for all tickets with destination MAD from `coupons150720.csv`.

You will need to extract ticket amounts with destination MAD, and then calculate:

1. Total ticket amounts per origin
2. Top 10 airlines by average amount

## Further Reading

https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html

https://www.datacamp.com/community/tutorials/apache-spark-python

https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

https://ogirardot.wordpress.com/2015/05/29/rdds-are-the-new-bytecode-of-apache-spark/

https://stackoverflow.com/questions/36822224/what-are-the-pros-and-cons-of-parquet-format-compared-to-other-formats

